In [ ]:
!pip install notebook
!pip install PyQt5
!pip install selenium
!pip install beautifulsoup4
!pip install nltk
!pip install wordcloud
!pip install matplotlib

import nltk
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QHBoxLayout, QWidget,
    QPushButton, QLabel, QLineEdit, QTextEdit, QMessageBox
)
from PyQt5.QtCore import pyqtSlot
from PyQt5.QtGui import QIcon
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas


class SentimentAnalysisApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.title = 'YouTube Sentiment Analysis'
        self.left = 100
        self.top = 100
        self.width = 1200
        self.height = 600
        self.initUI()

    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)

        layout = QVBoxLayout()

        self.youtube_link_label = QLabel('Enter YouTube link:')
        layout.addWidget(self.youtube_link_label)

        self.youtube_link_input = QLineEdit()
        layout.addWidget(self.youtube_link_input)

        self.run_analysis_button = QPushButton('Run Analysis')
        self.run_analysis_button.clicked.connect(self.onRunAnalysisClicked)
        layout.addWidget(self.run_analysis_button)

        self.output_text = QTextEdit()
        layout.addWidget(self.output_text)

        self.wordcloud_layout = QHBoxLayout()
        layout.addLayout(self.wordcloud_layout)

        self.general_wordcloud_text = QTextEdit()
        self.wordcloud_layout.addWidget(self.general_wordcloud_text)

        self.sentiment_wordcloud_text = QTextEdit()
        self.wordcloud_layout.addWidget(self.sentiment_wordcloud_text)

        self.negative_wordcloud_text = QTextEdit()
        self.wordcloud_layout.addWidget(self.negative_wordcloud_text)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)

    @pyqtSlot()
    def onRunAnalysisClicked(self):
        video_url = self.youtube_link_input.text()
        if video_url:
            comments = scrape_youtube_comments(video_url)
            self.output_text.setText("Analysis completed.\n")
            self.display_wordclouds(comments)
            self.display_sentiment_statistics(comments)
        else:
            QMessageBox.warning(self, 'Warning', 'Please enter a YouTube link.')

    def display_wordclouds(self, comments):
        self.display_wordcloud(comments, self.general_wordcloud_text, title='General Word Cloud')
        self.display_sentiment_wordcloud(comments, self.sentiment_wordcloud_text, title='Sentiment Assessment Word Cloud')
        self.display_negative_wordcloud(comments, self.negative_wordcloud_text, title='Word Cloud of Negative Comments')

    def display_wordcloud(self, comments, textedit_widget, title):
        all_comments = ' '.join(comments)
        wordcloud = WordCloud(width=400, height=400, background_color='white').generate(all_comments)
        self.plot_wordcloud(wordcloud, textedit_widget, title)

    def display_sentiment_wordcloud(self, comments, textedit_widget, title):
        vader = SentimentIntensityAnalyzer()
        word_sentiment_scores = {}
        for comment in comments:
            scores = vader.polarity_scores(comment)
            for word in word_tokenize(comment):
                if word.lower() not in stopwords.words('english'):
                    if word not in word_sentiment_scores:
                        word_sentiment_scores[word] = scores['compound']
                    else:
                        word_sentiment_scores[word] += scores['compound']
        wordcloud = WordCloud(width=400, height=400, background_color='white').generate_from_frequencies(word_sentiment_scores)
        self.plot_wordcloud(wordcloud, textedit_widget, title)

    def display_negative_wordcloud(self, comments, textedit_widget, title):
        vader = SentimentIntensityAnalyzer()
        negative_comments = [comment for comment in comments if classify_sentiment(vader, comment) == 'Negative']
        all_negative_comments = ' '.join(negative_comments)
        wordcloud = WordCloud(width=400, height=400, background_color='white').generate(all_negative_comments)
        self.plot_wordcloud(wordcloud, textedit_widget, title)

    def plot_wordcloud(self, wordcloud, textedit_widget, title):
        fig = plt.figure(figsize=(6, 6))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.title(title)
        plt.axis('off')
        canvas = FigureCanvas(fig)
        layout = QVBoxLayout()
        layout.addWidget(canvas)
        textedit_widget.setLayout(layout)
        plt.close()

    def display_sentiment_statistics(self, comments):
        vader = SentimentIntensityAnalyzer()
        sentiment_scores = [vader.polarity_scores(comment)['compound'] for comment in comments]
        average_compound = sum(sentiment_scores) / len(sentiment_scores)
        positive_comments = sum(1 for score in sentiment_scores if score > 0)
        negative_comments = sum(1 for score in sentiment_scores if score < 0)
        neutral_comments = sum(1 for score in sentiment_scores if score == 0)

        self.output_text.append("\nSentiment Statistics:")
        self.output_text.append(f"Average sentiment score: {average_compound:.2f}")
        self.output_text.append(f"Total number of positive comments: {positive_comments}")
        self.output_text.append(f"Total number of negative comments: {negative_comments}")
        self.output_text.append(f"Total number of neutral comments: {neutral_comments}")


def scrape_youtube_comments(video_url):
    comments = []
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(video_url)

    # Scroll to load more comments
    for _ in range(5):
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(2)

    # Parse the HTML content
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    comment_elements = soup.find_all('yt-formatted-string', {'id': 'content-text'})

    # Extract comments
    for comment in comment_elements:
        comments.append(comment.get_text(strip=True))

    return comments


def classify_sentiment(vader, comment):
    scores = vader.polarity_scores(comment)
    if scores['compound'] >= 0.05:
        return 'Positive'
    elif scores['compound'] <= -0.05:
        return 'Negative'
    else:
        return 'Neutral'


def main():
    app = QApplication(sys.argv)
    app.setWindowIcon(QIcon('icon.png'))  # You can replace 'icon.png' with the path to your icon file
    ex = SentimentAnalysisApp()
    ex.show()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()
